Import necessary libraries

In [78]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime
import multiprocessing
import os

Function to normalize column names and standardize 'gmt_time'

In [79]:
def normalize_column_names(df, filepath):
    df = df.copy()
    # Replace non-alphanumeric characters with underscores
    df.columns = df.columns.str.replace(r"[^\w\s]", "_", regex=True)
    # Replace any whitespace with a single underscore
    df.columns = df.columns.str.replace(r"\s+", "_", regex=True)
    # Convert to lowercase
    df.columns = df.columns.str.lower()
    # Replace multiple underscores with a single underscore
    df.columns = df.columns.str.replace(r"_+", "_", regex=True)
    # Remove leading and trailing underscores
    df.columns = df.columns.str.strip('_')
    # Map known time columns to 'gmt_time'
    time_column_variations = ['gmt_time', 'time', 'date_time', 'datetime', 'timestamp', 'gmt']
    found_time_column = False
    for col in time_column_variations:
        if col in df.columns:
            df.rename(columns={col: 'gmt_time'}, inplace=True)
            found_time_column = True
            break
    if not found_time_column:
        print(f"Warning: No recognized time column found in {filepath}. Please check the column names.")
    # Print columns for debugging
    print(f"Columns after normalization for {filepath}: {df.columns.tolist()}")
    return df

Function to load and preprocess a single dataset

In [80]:
def load_and_preprocess_dataset(filepath):
    df = pd.read_csv(filepath)
    df.replace('No Data Available', np.nan, inplace=True)
    df = normalize_column_names(df, filepath)
    if 'gmt_time' in df.columns:
        df['gmt_time'] = pd.to_datetime(df['gmt_time'], errors='coerce')
    else:
        raise KeyError(f"The required column 'gmt_time' is missing in the dataset {filepath}.")
    # Convert all columns except 'gmt_time' to numeric
    cols = df.columns.drop('gmt_time')
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    return df

Load and preprocess datasets using the defined function

In [ ]:
print("Loading and preprocessing datasets...")
balancing_data = load_and_preprocess_dataset('../data/balancing_data.csv')
demand_load_data = load_and_preprocess_dataset('../data/demand_load_data.csv')
generation_data = load_and_preprocess_dataset('../data/generation_data.csv')
price_data = load_and_preprocess_dataset('../data/price_data.csv')

Loading and preprocessing datasets...
Columns after normalization for /Users/sravansridhar/Desktop/Edp_datathon/data/balancing_data.csv: ['gmt_time', 'system_price_eso_outturn_gb_mwh', 'niv_outturn_ve_long_gb_mw', 'bm_bid_acceptances_total_gb_mw', 'bm_offer_acceptances_total_gb_mw', 'total_bsad_volume_turn_up_gb_mw', 'total_bsad_volume_turn_down_gb_mw', 'total_bsad_volume_total_gb_mw', 'intraday_volume_epex_outturn_apx_mid_gb_mwh']
Columns after normalization for /Users/sravansridhar/Desktop/Edp_datathon/data/demand_load_data.csv: ['gmt_time', 'loss_of_load_probability_latest_gb', 'actual_total_load_gb_mw', 'demand_outturn_itsdo_gb_mw']
Columns after normalization for /Users/sravansridhar/Desktop/Edp_datathon/data/generation_data.csv: ['gmt_time', 'actual_aggregated_generation_by_type_biomass_gb_mw', 'actual_aggregated_generation_by_type_fossil_gas_gb_mw', 'actual_aggregated_generation_by_type_fossil_hard_coal_gb_mw', 'actual_aggregated_generation_by_type_fossil_oil_gb_mw', 'actual_agg

Merge datasets on 'gmt_time'

In [82]:
print("Merging datasets...")
data_frames = [balancing_data, demand_load_data, generation_data, price_data]
merged_data = data_frames[0]
for df in data_frames[1:]:
    merged_data = pd.merge(merged_data, df, on='gmt_time', how='outer')

Merging datasets...


Handle missing values

In [83]:
print("Handling missing values...")
merged_data.sort_values('gmt_time', inplace=True)
merged_data.dropna(subset=['gmt_time'], inplace=True)
merged_data.fillna(method='ffill', inplace=True)
merged_data.fillna(method='bfill', inplace=True)
merged_data.dropna(inplace=True)

Handling missing values...


/var/folders/4n/xlvp5dd530zgvp5yzcjmsv400000gn/T/ipykernel_20997/630599973.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='ffill', inplace=True)
/var/folders/4n/xlvp5dd530zgvp5yzcjmsv400000gn/T/ipykernel_20997/630599973.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_data.fillna(method='bfill', inplace=True)


Ensure all data types are numeric except 'gmt_time'

In [84]:
print("Ensuring data types are correct...")
cols = merged_data.columns.drop('gmt_time')
merged_data[cols] = merged_data[cols].apply(pd.to_numeric, errors='coerce')

Ensuring data types are correct...


Feature engineering

In [85]:
print("Feature engineering...")
# Add time-based features
merged_data['hour'] = merged_data['gmt_time'].dt.hour
merged_data['day_of_week'] = merged_data['gmt_time'].dt.dayofweek
merged_data['month'] = merged_data['gmt_time'].dt.month
merged_data['is_weekend'] = (merged_data['day_of_week'] >= 5).astype(int)

Feature engineering...


Add lag features

In [86]:
print("Adding lag features...")
lag_features = ['system_price_eso_outturn_gb_mwh', 'niv_outturn_ve_long_gb_mw']
for lag in [1, 24, 48]:  # Lag by 1 hour, 1 day, 2 days
    for col in lag_features:
        if col in merged_data.columns:
            merged_data[f'{col}_lag_{lag}'] = merged_data[col].shift(lag)
        else:
            print(f"Warning: Column {col} not found in merged_data.")

Adding lag features...


Add rolling statistics

In [87]:
print("Adding rolling statistics...")
for window in [7, 30]:  # Weekly and monthly
    for col in lag_features:
        if col in merged_data.columns:
            merged_data[f'{col}_rolling_mean_{window}'] = merged_data[col].rolling(window).mean()
        else:
            print(f"Warning: Column {col} not found in merged_data.")

Adding rolling statistics...


Drop rows with NaN values created due to lags/rolling

In [88]:
merged_data.dropna(inplace=True)

Define features and targets

In [89]:
print("Defining features and targets...")
X = merged_data.drop(columns=['gmt_time', 'system_price_eso_outturn_gb_mwh', 'niv_outturn_ve_long_gb_mw'])
y_price = merged_data['system_price_eso_outturn_gb_mwh']
y_niv = merged_data['niv_outturn_ve_long_gb_mw']

Defining features and targets...


Ensure there are no remaining non-numeric columns

In [90]:
print("Verifying that all features are numeric...")
non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_cols:
    print(f"Converting non-numeric columns to numeric: {non_numeric_cols}")
    X[non_numeric_cols] = X[non_numeric_cols].apply(pd.to_numeric, errors='coerce')
    X.dropna(inplace=True)
    y_price = y_price.loc[X.index]
    y_niv = y_niv.loc[X.index]

Verifying that all features are numeric...


Implement a pipeline with scaling

In [91]:
print("Creating pipelines...")
from sklearn.preprocessing import StandardScaler

Creating pipelines...


Define the preprocessing and modeling pipeline

In [92]:
def create_pipeline(model):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    return pipeline

Define base models

In [93]:
print("Defining base models...")
# XGBoost parameters (CPU version)
xgb_params = {
    'random_state': 42,
    'objective': 'reg:squarederror',
    'n_jobs': multiprocessing.cpu_count()
}

Defining base models...


LightGBM parameters (CPU version)

In [94]:
lgbm_params = {
    'random_state': 42,
    'n_jobs': multiprocessing.cpu_count()
}

In [95]:
base_models = [
    ('rf', RandomForestRegressor(random_state=42, n_jobs=multiprocessing.cpu_count())),
    ('gb', GradientBoostingRegressor(random_state=42)),
    ('xgb', XGBRegressor(**xgb_params)),
    ('lgbm', LGBMRegressor(**lgbm_params))
]

Create pipelines for base models

In [96]:
base_pipelines = [(name, create_pipeline(model)) for name, model in base_models]

Define stacking regressors

In [97]:
print("Defining stacking regressors...")
stacking_regressor_price = StackingRegressor(
    estimators=base_pipelines,
    final_estimator=Ridge(),
    passthrough=True,
    n_jobs=multiprocessing.cpu_count()
)
stacking_regressor_niv = StackingRegressor(
    estimators=base_pipelines,
    final_estimator=Ridge(),
    passthrough=True,
    n_jobs=multiprocessing.cpu_count()
)

Defining stacking regressors...


Hyperparameter tuning using RandomizedSearchCV

In [98]:
print("Starting hyperparameter tuning for System Price model...")
param_distributions = {
    'rf__model__n_estimators': [100, 200],
    'rf__model__max_depth': [None, 10],
    'gb__model__n_estimators': [100, 200],
    'gb__model__learning_rate': [0.05, 0.1],
    'xgb__model__n_estimators': [100, 200],
    'xgb__model__learning_rate': [0.05, 0.1],
    'lgbm__model__n_estimators': [100, 200],
    'lgbm__model__learning_rate': [0.05, 0.1],
    'final_estimator__alpha': [0.1, 1.0, 10.0]
}

Starting hyperparameter tuning for System Price model...


Function to perform RandomizedSearchCV with multiprocessing

In [99]:
def perform_random_search(model, X, y, param_distributions, model_name):
    print(f"Performing hyperparameter tuning for {model_name}...")
    tscv = TimeSeriesSplit(n_splits=3)
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=20,
        cv=tscv,
        scoring='neg_root_mean_squared_error',
        verbose=2,
        n_jobs=multiprocessing.cpu_count(),
        random_state=42
    )
    random_search.fit(X, y)
    best_model = random_search.best_estimator_
    print(f"Best parameters for {model_name}: {random_search.best_params_}")
    # Save the best model
    joblib.dump(best_model, f'pipeline_model_{model_name}.pkl')
    return best_model

Perform hyperparameter tuning for System Price

In [100]:
best_model_price = perform_random_search(
    stacking_regressor_price,
    X,
    y_price,
    param_distributions,
    'price'
)

Performing hyperparameter tuning for price...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005502 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7441
[LightGBM] [Info] Number of data points in the train set: 29568, number of used features: 33
[LightGBM] [Info] Start training from score 51.665461
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7462
[LightGBM] [Info] Number of data points in the train set: 88704, number of used features: 34
[LightGBM] [Info] Start training from score 90.056804
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003689 seconds.
You can set `force_col_wise=true` to re

/opt/homebrew/Caskroom/miniconda/base/envs/ds_env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008800 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7457
[LightGBM] [Info] Number of data points in the train set: 59136, number of used features: 34
[LightGBM] [Info] Start training from score 47.174889
[CV] END final_estimator__alpha=10.0, gb__model__learning_rate=0.05, gb__model__n_estimators=100, lgbm__model__learning_rate=0.1, lgbm__model__n_estimators=200, rf__model__max_depth=10, rf__model__n_estimators=100, xgb__model__learning_rate=0.05, xgb__model__n_estimators=100; total time=16.9min
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020599 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7462
[LightGBM] [Info] Number of data points in the train set: 88704, number of used features: 34
[LightGB

Perform hyperparameter tuning for NIV

In [101]:
print("Starting hyperparameter tuning for NIV model...")
best_model_niv = perform_random_search(
    stacking_regressor_niv,
    X,
    y_niv,
    param_distributions,
    'niv'
)

Starting hyperparameter tuning for NIV model...
Performing hyperparameter tuning for niv...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7441
[LightGBM] [Info] Number of data points in the train set: 29568, number of used features: 33
[LightGBM] [Info] Start training from score 133.659058
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005187 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7457
[LightGBM] [Info] Number of data points in the train set: 59136, number of used features: 34
[LightGBM] [Info] Start training from score 29.661958
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001459 s

/opt/homebrew/Caskroom/miniconda/base/envs/ds_env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004852 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7457
[LightGBM] [Info] Number of data points in the train set: 59136, number of used features: 34
[LightGBM] [Info] Start training from score 29.661958
[CV] END final_estimator__alpha=10.0, gb__model__learning_rate=0.05, gb__model__n_estimators=200, lgbm__model__learning_rate=0.1, lgbm__model__n_estimators=200, rf__model__max_depth=None, rf__model__n_estimators=100, xgb__model__learning_rate=0.05, xgb__model__n_estimators=100; total time=20.2min
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005440 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7462
[LightGBM] [Info] Number of data po

Prepare features for October 1, 2024

In [102]:
print("Preparing features for October 1, 2024...")
# Generate date range for October 1, 2024
dates_oct1 = pd.date_range(start='2024-10-01 00:00:00', end='2024-10-01 23:30:00', freq='30T')
october_1_features = pd.DataFrame({'gmt_time': dates_oct1})

Preparing features for October 1, 2024...


/var/folders/4n/xlvp5dd530zgvp5yzcjmsv400000gn/T/ipykernel_20997/2431054851.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_oct1 = pd.date_range(start='2024-10-01 00:00:00', end='2024-10-01 23:30:00', freq='30T')


Create the same features as in training

In [103]:
print("Creating features for October 1, 2024...")
october_1_features['hour'] = october_1_features['gmt_time'].dt.hour
october_1_features['day_of_week'] = october_1_features['gmt_time'].dt.dayofweek
october_1_features['month'] = october_1_features['gmt_time'].dt.month
october_1_features['is_weekend'] = (october_1_features['day_of_week'] >= 5).astype(int)

Creating features for October 1, 2024...


Estimate lag and rolling features using historical data

In [104]:
print("Estimating lag and rolling features using historical data...")
# Calculate historical averages for lag and rolling features
historical_data = merged_data.copy()
historical_data['time'] = historical_data['gmt_time'].dt.time
october_1_features['time'] = october_1_features['gmt_time'].dt.time

Estimating lag and rolling features using historical data...


List of features to estimate

In [105]:
features_to_estimate = X.columns.tolist()

Calculate mean values by time for features

In [106]:
mean_features = historical_data.groupby('time')[features_to_estimate].mean().reset_index()

Merge mean features with october_1_features

In [107]:
october_1_features = pd.merge(october_1_features, mean_features, on='time', how='left')

Drop unnecessary columns

In [108]:
october_1_features.drop(columns=['time', 'gmt_time'], inplace=True)

Ensure no missing values in features

In [109]:
print("Checking for missing values in October 1 features...")
if october_1_features.isnull().values.any():
    print("Filling missing values with historical means...")
    october_1_features.fillna(october_1_features.mean(), inplace=True)

Checking for missing values in October 1 features...


Predict for October 1, 2024

In [110]:
print("Predicting for October 1, 2024...")
# Load the pipelines
best_model_price_loaded = joblib.load('pipeline_model_price.pkl')
best_model_niv_loaded = joblib.load('pipeline_model_niv.pkl')

Predicting for October 1, 2024...


Create output DataFrame

In [114]:
# After defining X during training
training_feature_columns = X.columns.tolist()
print("Training Features:", training_feature_columns)


Training Features: ['bm_bid_acceptances_total_gb_mw', 'bm_offer_acceptances_total_gb_mw', 'total_bsad_volume_turn_up_gb_mw', 'total_bsad_volume_turn_down_gb_mw', 'total_bsad_volume_total_gb_mw', 'intraday_volume_epex_outturn_apx_mid_gb_mwh', 'loss_of_load_probability_latest_gb', 'actual_total_load_gb_mw', 'demand_outturn_itsdo_gb_mw', 'actual_aggregated_generation_by_type_biomass_gb_mw', 'actual_aggregated_generation_by_type_fossil_gas_gb_mw', 'actual_aggregated_generation_by_type_fossil_hard_coal_gb_mw', 'actual_aggregated_generation_by_type_fossil_oil_gb_mw', 'actual_aggregated_generation_by_type_hydro_pumped_storage_gb_mw', 'actual_aggregated_generation_by_type_hydro_run_of_river_and_poundage_gb_mw', 'actual_aggregated_generation_by_type_nuclear_gb_mw', 'actual_aggregated_generation_by_type_solar_gb_mw', 'actual_aggregated_generation_by_type_wind_onshore_gb_mw', 'actual_aggregated_generation_by_type_wind_offshore_gb_mw', 'day_ahead_price_epex_half_hourly_local_gb_lc_mwh', 'intraday_

In [115]:
training_feature_columns = [
    'bm_bid_acceptances_total_gb_mw',
    'bm_offer_acceptances_total_gb_mw',
    'total_bsad_volume_turn_up_gb_mw',
    'total_bsad_volume_turn_down_gb_mw',
    'total_bsad_volume_total_gb_mw',
    'intraday_volume_epex_outturn_apx_mid_gb_mwh',
    'loss_of_load_probability_latest_gb',
    'actual_total_load_gb_mw',
    'demand_outturn_itsdo_gb_mw',
    'actual_aggregated_generation_by_type_biomass_gb_mw',
    'actual_aggregated_generation_by_type_fossil_gas_gb_mw',
    'actual_aggregated_generation_by_type_fossil_hard_coal_gb_mw',
    'actual_aggregated_generation_by_type_fossil_oil_gb_mw',
    'actual_aggregated_generation_by_type_hydro_pumped_storage_gb_mw',
    'actual_aggregated_generation_by_type_hydro_run_of_river_and_poundage_gb_mw',
    'actual_aggregated_generation_by_type_nuclear_gb_mw',
    'actual_aggregated_generation_by_type_solar_gb_mw',
    'actual_aggregated_generation_by_type_wind_onshore_gb_mw',
    'actual_aggregated_generation_by_type_wind_offshore_gb_mw',
    'day_ahead_price_epex_half_hourly_local_gb_lc_mwh',
    'intraday_price_epex_outturn_apx_mid_gb_mwh',
    'hour',
    'day_of_week',
    'month',
    'is_weekend',
    'system_price_eso_outturn_gb_mwh_lag_1',
    'niv_outturn_ve_long_gb_mw_lag_1',
    'system_price_eso_outturn_gb_mwh_lag_24',
    'niv_outturn_ve_long_gb_mw_lag_24',
    'system_price_eso_outturn_gb_mwh_lag_48',
    'niv_outturn_ve_long_gb_mw_lag_48',
    'system_price_eso_outturn_gb_mwh_rolling_mean_7',
    'niv_outturn_ve_long_gb_mw_rolling_mean_7',
    'system_price_eso_outturn_gb_mwh_rolling_mean_30',
    'niv_outturn_ve_long_gb_mw_rolling_mean_30'
]


In [116]:
# Generate date range for October 1, 2024 at 30-minute intervals
dates_oct1 = pd.date_range(start='2024-10-01 00:00:00', end='2024-10-01 23:30:00', freq='30T')
october_1_features = pd.DataFrame({'gmt_time': dates_oct1})

# Create time-based features
october_1_features['hour'] = october_1_features['gmt_time'].dt.hour
october_1_features['day_of_week'] = october_1_features['gmt_time'].dt.dayofweek
october_1_features['month'] = october_1_features['gmt_time'].dt.month
october_1_features['is_weekend'] = (october_1_features['day_of_week'] >= 5).astype(int)


/var/folders/4n/xlvp5dd530zgvp5yzcjmsv400000gn/T/ipykernel_20997/792563929.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates_oct1 = pd.date_range(start='2024-10-01 00:00:00', end='2024-10-01 23:30:00', freq='30T')


In [117]:
# Calculate historical means from the training data
historical_means = merged_data.mean()

# Define lag and rolling feature columns
lag_features = ['system_price_eso_outturn_gb_mwh', 'niv_outturn_ve_long_gb_mw']
lag_feature_cols = [f'{col}_lag_{lag}' for col in lag_features for lag in [1, 24, 48]]
rolling_feature_cols = [f'{col}_rolling_mean_{window}' for col in lag_features for window in [7, 30]]

# Assign historical mean values to lag features
for col in lag_feature_cols:
    october_1_features[col] = historical_means.get(col, np.nan)  # Use np.nan if the mean is not available

# Assign historical mean values to rolling features
for col in rolling_feature_cols:
    october_1_features[col] = historical_means.get(col, np.nan)  # Use np.nan if the mean is not available


In [118]:
# List of original features from merged datasets used during training
original_features = [
    'bm_bid_acceptances_total_gb_mw',
    'bm_offer_acceptances_total_gb_mw',
    'total_bsad_volume_turn_up_gb_mw',
    'total_bsad_volume_turn_down_gb_mw',
    'total_bsad_volume_total_gb_mw',
    'intraday_volume_epex_outturn_apx_mid_gb_mwh',
    'loss_of_load_probability_latest_gb',
    'actual_total_load_gb_mw',
    'demand_outturn_itsdo_gb_mw',
    'actual_aggregated_generation_by_type_biomass_gb_mw',
    'actual_aggregated_generation_by_type_fossil_gas_gb_mw',
    'actual_aggregated_generation_by_type_fossil_hard_coal_gb_mw',
    'actual_aggregated_generation_by_type_fossil_oil_gb_mw',
    'actual_aggregated_generation_by_type_hydro_pumped_storage_gb_mw',
    'actual_aggregated_generation_by_type_hydro_run_of_river_and_poundage_gb_mw',
    'actual_aggregated_generation_by_type_nuclear_gb_mw',
    'actual_aggregated_generation_by_type_solar_gb_mw',
    'actual_aggregated_generation_by_type_wind_onshore_gb_mw',
    'actual_aggregated_generation_by_type_wind_offshore_gb_mw',
    'day_ahead_price_epex_half_hourly_local_gb_lc_mwh',
    'intraday_price_epex_outturn_apx_mid_gb_mwh'
]

# Assign historical mean values to original features
for col in original_features:
    october_1_features[col] = historical_means.get(col, np.nan)  # Use np.nan if the mean is not available


In [119]:
# Check for missing values
if october_1_features.isnull().values.any():
    october_1_features.fillna(october_1_features.mean(), inplace=True)
    print("Filled missing values with historical means.")


In [120]:
# Drop 'gmt_time' as it's not a feature used in the model
october_1_features = october_1_features.drop(columns=['gmt_time'])

# Ensure the order of columns matches the training data
training_feature_columns = X.columns.tolist()
october_1_features = october_1_features[training_feature_columns]


In [121]:
# Verify that all training features are present in the prediction data
missing_features = set(training_feature_columns) - set(october_1_features.columns)
extra_features = set(october_1_features.columns) - set(training_feature_columns)

if missing_features:
    print(f"Missing features in prediction data: {missing_features}")
    # Assign default values or handle as needed
    for feature in missing_features:
        october_1_features[feature] = historical_means.get(feature, 0)
        print(f"Assigned default value 0 to missing feature: {feature}")

if extra_features:
    print(f"Extra features in prediction data: {extra_features}")
    # Remove extra features if they are not needed
    october_1_features = october_1_features.drop(columns=extra_features)
    print(f"Dropped extra features: {extra_features}")

# Final verification
print("Final feature set for prediction:")
print(october_1_features.columns.tolist())


Final feature set for prediction:
['bm_bid_acceptances_total_gb_mw', 'bm_offer_acceptances_total_gb_mw', 'total_bsad_volume_turn_up_gb_mw', 'total_bsad_volume_turn_down_gb_mw', 'total_bsad_volume_total_gb_mw', 'intraday_volume_epex_outturn_apx_mid_gb_mwh', 'loss_of_load_probability_latest_gb', 'actual_total_load_gb_mw', 'demand_outturn_itsdo_gb_mw', 'actual_aggregated_generation_by_type_biomass_gb_mw', 'actual_aggregated_generation_by_type_fossil_gas_gb_mw', 'actual_aggregated_generation_by_type_fossil_hard_coal_gb_mw', 'actual_aggregated_generation_by_type_fossil_oil_gb_mw', 'actual_aggregated_generation_by_type_hydro_pumped_storage_gb_mw', 'actual_aggregated_generation_by_type_hydro_run_of_river_and_poundage_gb_mw', 'actual_aggregated_generation_by_type_nuclear_gb_mw', 'actual_aggregated_generation_by_type_solar_gb_mw', 'actual_aggregated_generation_by_type_wind_onshore_gb_mw', 'actual_aggregated_generation_by_type_wind_offshore_gb_mw', 'day_ahead_price_epex_half_hourly_local_gb_lc_m

In [122]:
# Load the saved pipelines
best_model_price_loaded = joblib.load('pipeline_model_price.pkl')
best_model_niv_loaded = joblib.load('pipeline_model_niv.pkl')

# Make predictions
october_1_predictions_price = best_model_price_loaded.predict(october_1_features)
october_1_predictions_niv = best_model_niv_loaded.predict(october_1_features)

# Create output DataFrame
output = pd.DataFrame({
    'GTM_TIME': dates_oct1,
    'SYSTEM_PRICE': october_1_predictions_price,
    'NIV_OUTTURN': october_1_predictions_niv
})

# Save predictions to CSV
output.to_csv('predictions_october_1_2024.csv', index=False)
print("Predictions saved to 'predictions_october_1_2024.csv'.")


Predictions saved to 'predictions_october_1_2024.csv'.


In [ ]:
print("Saving predictions...")
output.to_csv('predictions_october_1_2024.csv', index=False)
print("Predictions saved to 'predictions_october_1_2024.csv'.")

Evaluate model performance on the training set (optional)

In [123]:
print("Evaluating model performance on training data...")
y_pred_price = best_model_price_loaded.predict(X)
rmse_price = mean_squared_error(y_price, y_pred_price, squared=False)
print(f"Training RMSE for System Price: {rmse_price:.4f}")

Evaluating model performance on training data...
Training RMSE for System Price: 36.8048


/opt/homebrew/Caskroom/miniconda/base/envs/ds_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [124]:
y_pred_niv = best_model_niv_loaded.predict(X)
rmse_niv = mean_squared_error(y_niv, y_pred_niv, squared=False)
print(f"Training RMSE for NIV: {rmse_niv:.4f}")

Training RMSE for NIV: 72.8251


/opt/homebrew/Caskroom/miniconda/base/envs/ds_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Save the merged data and features for Streamlit app

In [125]:
import os
import shutil

# Define the name of the destination folder
destination_folder = 'streamlit_assets'

# Get the current working directory
current_dir = os.getcwd()

# Define the full path for the destination folder
destination_path = os.path.join(current_dir, destination_folder)

# List of files to copy to the Streamlit assets folder
files_to_copy = [
    'pipeline_model_price.pkl',
    'pipeline_model_niv.pkl',
    'merged_data_processed.csv',
    'predictions_october_1_2024.csv',
    'features.csv',
    'target_price.csv',
    'target_niv.csv',
    # Add any additional files here
    # 'additional_file.ext',
]

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_path):
    os.makedirs(destination_path)
    print(f"Created folder: {destination_path}")
else:
    print(f"Folder already exists: {destination_path}")

# Copy each file to the destination folder
for file in files_to_copy:
    source_file = os.path.join(current_dir, file)
    destination_file = os.path.join(destination_path, file)
    
    if os.path.exists(source_file):
        shutil.copy(source_file, destination_file)
        print(f"Copied: {file} to {destination_folder}/")
    else:
        print(f"Warning: {file} does not exist and was not copied.")


Created folder: /Users/sravansridhar/Desktop/Edp_datathon/notebooks/streamlit_assets
Copied: pipeline_model_price.pkl to streamlit_assets/
Copied: pipeline_model_niv.pkl to streamlit_assets/
Copied: predictions_october_1_2024.csv to streamlit_assets/


In [130]:
import os
import shutil
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, make_scorer

# ... [Your existing data processing and model training code] ...

# After training your models and creating 'merged_data' DataFrame

# 1. Ensure 'merged_data' exists
if 'merged_data' in locals():
    print("merged_data DataFrame is ready.")
else:
    raise NameError("merged_data DataFrame is not found. Please ensure it is created before saving.")

# 2. Create 'streamlit_assets' folder
destination_folder = 'streamlit_assets'
current_dir = os.getcwd()
destination_path = os.path.join(current_dir, destination_folder)

if not os.path.exists(destination_path):
    os.makedirs(destination_path)
    print(f"Created folder: {destination_path}")
else:
    print(f"Folder already exists: {destination_path}")

# 3. Save 'merged_data_processed.csv'
merged_data_path = os.path.join(destination_path, 'merged_data_processed.csv')
merged_data.to_csv(merged_data_path, index=False)
print(f"Saved 'merged_data_processed.csv' to '{destination_folder}/' folder.")

# 4. Save additional assets
files_to_copy = [
    'pipeline_model_price.pkl',
    'pipeline_model_niv.pkl',
    'predictions_october_1_2024.csv',
    'features.csv',
    'target_price.csv',
    'target_niv.csv',
    # Add any additional files here
    # 'additional_file.ext',
]

for file in files_to_copy:
    source_file = os.path.join(current_dir, file)
    destination_file = os.path.join(destination_path, file)
    
    if os.path.exists(source_file):
        shutil.copy(source_file, destination_file)
        print(f"Copied: {file} to '{destination_folder}/'")
    else:
        print(f"Warning: {file} does not exist and was not copied.")


merged_data DataFrame is ready.
Created folder: /Users/sravansridhar/Desktop/Edp_datathon/notebooks/streamlit_assets
Saved 'merged_data_processed.csv' to 'streamlit_assets/' folder.
Copied: pipeline_model_price.pkl to 'streamlit_assets/'
Copied: pipeline_model_niv.pkl to 'streamlit_assets/'
Copied: predictions_october_1_2024.csv to 'streamlit_assets/'


In [134]:
import os
import shutil
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, make_scorer

# ... [Your existing data processing and model training code] ...

# After training your models and creating 'merged_data' DataFrame

# 1. Ensure 'merged_data' DataFrame exists
if 'merged_data' in locals():
    print("merged_data DataFrame is ready.")
else:
    raise NameError("merged_data DataFrame is not found. Please ensure it is created before saving.")

# 2. Define features (X) and targets (y_price and y_niv)
X = merged_data.drop(columns=['gmt_time', 'system_price_eso_outturn_gb_mwh', 'niv_outturn_ve_long_gb_mw'])
y_price = merged_data['system_price_eso_outturn_gb_mwh']
y_niv = merged_data['niv_outturn_ve_long_gb_mw']

# 3. Verify that all columns in X are numeric
non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_cols:
    print(f"Non-numeric columns found: {non_numeric_cols}")
    # Handle non-numeric columns if any, e.g., encoding or dropping
    # For simplicity, let's drop them
    X = X.drop(columns=non_numeric_cols)
    print(f"Dropped non-numeric columns: {non_numeric_cols}")

# 4. Create the 'streamlit_assets' folder if it doesn't exist
destination_folder = 'streamlit_assets'
current_dir = os.getcwd()
destination_path = os.path.join(current_dir, destination_folder)

if not os.path.exists(destination_path):
    os.makedirs(destination_path)
    print(f"Created folder: {destination_path}")
else:
    print(f"Folder already exists: {destination_path}")

# 5. Save 'merged_data_processed.csv' with only features
merged_data_processed_path = os.path.join(destination_path, 'merged_data_processed.csv')
X.to_csv(merged_data_processed_path, index=False)
print(f"Saved 'merged_data_processed.csv' to '{destination_folder}/' folder.")

# 6. Save target variables
y_price_path = os.path.join(destination_path, 'target_price.csv')
y_niv_path = os.path.join(destination_path, 'target_niv.csv')

y_price.to_csv(y_price_path, index=False)
y_niv.to_csv(y_niv_path, index=False)

print(f"Saved target variables to '{destination_folder}/' folder.")

# 7. (Optional) Save additional assets like trained models
files_to_copy = [
    'pipeline_model_price.pkl',
    'pipeline_model_niv.pkl',
    'predictions_october_1_2024.csv',
    # Add any additional files here
]

for file in files_to_copy:
    source_file = os.path.join(current_dir, file)
    destination_file = os.path.join(destination_path, file)
    
    if os.path.exists(source_file):
        shutil.copy(source_file, destination_file)
        print(f"Copied: {file} to '{destination_folder}/'")
    else:
        print(f"Warning: {file} does not exist and was not copied.")


merged_data DataFrame is ready.
Folder already exists: /Users/sravansridhar/Desktop/Edp_datathon/notebooks/streamlit_assets
Saved 'merged_data_processed.csv' to 'streamlit_assets/' folder.
Saved target variables to 'streamlit_assets/' folder.
Copied: pipeline_model_price.pkl to 'streamlit_assets/'
Copied: pipeline_model_niv.pkl to 'streamlit_assets/'
Copied: predictions_october_1_2024.csv to 'streamlit_assets/'


In [135]:
import os
import shutil
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, make_scorer

# ... [Your existing data processing and model training code] ...

# After training your models and creating 'merged_data' DataFrame

# 1. Ensure 'merged_data' DataFrame exists
if 'merged_data' in locals():
    print("merged_data DataFrame is ready.")
else:
    raise NameError("merged_data DataFrame is not found. Please ensure it is created before saving.")

# 2. Define features (X) and targets (y_price and y_niv)
X = merged_data.drop(columns=['gmt_time', 'system_price_eso_outturn_gb_mwh', 'niv_outturn_ve_long_gb_mw'])
y_price = merged_data['system_price_eso_outturn_gb_mwh']
y_niv = merged_data['niv_outturn_ve_long_gb_mw']

# 3. Verify that all columns in X are numeric
non_numeric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_cols:
    print(f"Non-numeric columns found: {non_numeric_cols}")
    # Handle non-numeric columns if any, e.g., encoding or dropping
    # For simplicity, let's drop them
    X = X.drop(columns=non_numeric_cols)
    print(f"Dropped non-numeric columns: {non_numeric_cols}")

# 4. Create the 'streamlit_assets' folder if it doesn't exist
destination_folder = 'streamlit_assets'
current_dir = os.getcwd()
destination_path = os.path.join(current_dir, destination_folder)

if not os.path.exists(destination_path):
    os.makedirs(destination_path)
    print(f"Created folder: {destination_path}")
else:
    print(f"Folder already exists: {destination_path}")

# 5. Save 'merged_data_processed.csv' with only features
merged_data_processed_path = os.path.join(destination_path, 'merged_data_processed.csv')
X.to_csv(merged_data_processed_path, index=False)
print(f"Saved 'merged_data_processed.csv' to '{destination_folder}/' folder.")

# 6. Save target variables
y_price_path = os.path.join(destination_path, 'target_price.csv')
y_niv_path = os.path.join(destination_path, 'target_niv.csv')

y_price.to_csv(y_price_path, index=False)
y_niv.to_csv(y_niv_path, index=False)

print(f"Saved target variables to '{destination_folder}/' folder.")

# 7. (Optional) Save additional assets like trained models
files_to_copy = [
    'pipeline_model_price.pkl',
    'pipeline_model_niv.pkl',
    'predictions_october_1_2024.csv',
    # Add any additional files here
]

for file in files_to_copy:
    source_file = os.path.join(current_dir, file)
    destination_file = os.path.join(destination_path, file)
    
    if os.path.exists(source_file):
        shutil.copy(source_file, destination_file)
        print(f"Copied: {file} to '{destination_folder}/'")
    else:
        print(f"Warning: {file} does not exist and was not copied.")


merged_data DataFrame is ready.
Folder already exists: /Users/sravansridhar/Desktop/Edp_datathon/notebooks/streamlit_assets
Saved 'merged_data_processed.csv' to 'streamlit_assets/' folder.
Saved target variables to 'streamlit_assets/' folder.
Copied: pipeline_model_price.pkl to 'streamlit_assets/'
Copied: pipeline_model_niv.pkl to 'streamlit_assets/'
Copied: predictions_october_1_2024.csv to 'streamlit_assets/'


In [136]:
import pandas as pd
import os

# Define the path to the Streamlit assets folder
assets_path = 'streamlit_assets'

# Verify 'merged_data_processed.csv'
merged_data_processed_path = os.path.join(assets_path, 'merged_data_processed.csv')
merged_data_processed = pd.read_csv(merged_data_processed_path)

# Check if 'gmt_time' is present
if 'gmt_time' in merged_data_processed.columns:
    print("Error: 'gmt_time' column is present in 'merged_data_processed.csv'. It should be excluded.")
else:
    print("'gmt_time' column successfully excluded from 'merged_data_processed.csv'.")

# Verify that all columns are numeric
non_numeric_cols = merged_data_processed.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric_cols:
    print(f"Non-numeric columns found: {non_numeric_cols}")
else:
    print("All columns in 'merged_data_processed.csv' are numeric.")

# Verify target files
y_price_path = os.path.join(assets_path, 'target_price.csv')
y_niv_path = os.path.join(assets_path, 'target_niv.csv')

if os.path.exists(y_price_path):
    y_price = pd.read_csv(y_price_path)
    print("'target_price.csv' loaded successfully.")
else:
    print("Error: 'target_price.csv' not found.")

if os.path.exists(y_niv_path):
    y_niv = pd.read_csv(y_niv_path)
    print("'target_niv.csv' loaded successfully.")
else:
    print("Error: 'target_niv.csv' not found.")


'gmt_time' column successfully excluded from 'merged_data_processed.csv'.
All columns in 'merged_data_processed.csv' are numeric.
'target_price.csv' loaded successfully.
'target_niv.csv' loaded successfully.


In [137]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import cross_validate, TimeSeriesSplit
from sklearn.metrics import make_scorer, mean_squared_error
import os

# Define a function to calculate RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Create a scorer for RMSE
rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Define the path to the Streamlit assets folder
assets_path = 'streamlit_assets'

# Load the preprocessed features
data_path = os.path.join(assets_path, 'merged_data_processed.csv')
if os.path.exists(data_path):
    X = pd.read_csv(data_path)
    print("Loaded 'merged_data_processed.csv' successfully.")
else:
    raise FileNotFoundError(f"'{data_path}' not found. Please ensure it is saved correctly.")

# Load target variables
y_price_path = os.path.join(assets_path, 'target_price.csv')
y_niv_path = os.path.join(assets_path, 'target_niv.csv')

if os.path.exists(y_price_path):
    y_price = pd.read_csv(y_price_path)['system_price_eso_outturn_gb_mwh']
    print("Loaded 'target_price.csv' successfully.")
else:
    raise FileNotFoundError(f"'{y_price_path}' not found. Please ensure it is saved correctly.")

if os.path.exists(y_niv_path):
    y_niv = pd.read_csv(y_niv_path)['niv_outturn_ve_long_gb_mw']
    print("Loaded 'target_niv.csv' successfully.")
else:
    raise FileNotFoundError(f"'{y_niv_path}' not found. Please ensure it is saved correctly.")

# Load the trained models
model_price_path = os.path.join(assets_path, 'pipeline_model_price.pkl')
model_niv_path = os.path.join(assets_path, 'pipeline_model_niv.pkl')

if os.path.exists(model_price_path):
    best_model_price_loaded = joblib.load(model_price_path)
    print("Loaded 'pipeline_model_price.pkl' successfully.")
else:
    raise FileNotFoundError(f"'{model_price_path}' not found. Please ensure it is saved correctly.")

if os.path.exists(model_niv_path):
    best_model_niv_loaded = joblib.load(model_niv_path)
    print("Loaded 'pipeline_model_niv.pkl' successfully.")
else:
    raise FileNotFoundError(f"'{model_niv_path}' not found. Please ensure it is saved correctly.")

# Define the cross-validation strategy
tscv = TimeSeriesSplit(n_splits=5)

# Function to evaluate a model and print RMSE scores
def evaluate_model(model, X, y, model_name):
    print(f"\nEvaluating {model_name} Model:")
    
    try:
        # Perform cross-validation
        cv_results = cross_validate(
            model, 
            X, 
            y, 
            cv=tscv, 
            scoring=rmse_scorer, 
            return_train_score=True,
            n_jobs=-1,  # Utilize all available CPU cores
            error_score='raise'  # Raise exceptions for debugging
        )
        
        # Extract training and validation RMSE scores
        train_rmse = -cv_results['train_score']  # Multiply by -1 to get positive RMSE
        val_rmse = -cv_results['test_score']
        
        # Calculate mean and standard deviation
        train_rmse_mean = train_rmse.mean()
        train_rmse_std = train_rmse.std()
        val_rmse_mean = val_rmse.mean()
        val_rmse_std = val_rmse.std()
        
        # Print the results
        print(f"Training RMSE: {train_rmse_mean:.4f} ± {train_rmse_std:.4f}")
        print(f"Validation RMSE: {val_rmse_mean:.4f} ± {val_rmse_std:.4f}")
    
    except Exception as e:
        print(f"Error during cross-validation for {model_name}: {e}")

# Evaluate the System Price model
evaluate_model(best_model_price_loaded, X, y_price, 'System Price')

# Evaluate the NIV model
evaluate_model(best_model_niv_loaded, X, y_niv, 'NIV')


Loaded 'merged_data_processed.csv' successfully.
Loaded 'target_price.csv' successfully.
Loaded 'target_niv.csv' successfully.
Loaded 'pipeline_model_price.pkl' successfully.
Loaded 'pipeline_model_niv.pkl' successfully.

Evaluating System Price Model:
[LightGBM] [Info] [LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009964 seconds.
You can set `force_col_wise=true` to remove the overhead.
Auto-choosing col-wise multi-threading, the overhead of testing was 0.009515 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7450
[LightGBM] [Info] Total Bins 7424
[LightGBM] [Info] Number of data points in the train set: 39424, number of used features: 34
[LightGBM] [Info] Number of data points in the train set: 19712, number of used features: 33
[LightGBM] [Info] [LightGBM] [Info] Start training from score 57.334813Start training from score 47.721245

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the